In [ ]:
import cebra
import torch
import torch.utils
import pandas as pd
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from utils import TensorDataset, SimpleTensorDataset, SupervisedNNSolver, LazySimpleTensorDataset
import dotenv
import os
dotenv.load_dotenv()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

/home/icaro/Documents/doctorate/dimensionality-reduction/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [3]:
from pyspark.sql import SparkSession

# Initialize a SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("LocalCluster") \
    .config("spark.driver.memory", "10g") \
    .config("spark.executor.memory", "10g") \
    .getOrCreate()

# Verify the SparkContext
print(spark.sparkContext.getConf().getAll())

24/12/14 11:05:36 WARN Utils: Your hostname, aigo resolves to a loopback address: 127.0.1.1; using 131.220.127.56 instead (on interface eno1)
24/12/14 11:05:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/14 11:05:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


[('spark.app.startTime', '1734185136737'), ('spark.driver.extraJavaOptions', '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'), ('spark.driver.host', '131.220.127.56'), ('spark.executor.memor

# Data Load

In [4]:
# Load parquet files from the folder given by the environment variable $DATA_PATH
parquet_dir = os.environ.get("DATA_PATH")
df = spark.read.format("parquet").load(parquet_dir)
df.show(5)

+-------------------+-----------------+-----------------+----+--------------------+--------------------+-------------------+------------------+------------+--------------------+--------------------+
|               time|         position|original_velocity|pump|         neural_data|                  dt|                 dx|          velocity|       index| positional_encoding|           file_name|
+-------------------+-----------------+-----------------+----+--------------------+--------------------+-------------------+------------------+------------+--------------------+--------------------+
|                0.0|340.5632161968261|14.70307693342713| 0.0|[0.01222460158169...|                 0.0|  340.5632161968261|               0.0|128849018880|0.016677520715069446|ID18150/Day2/Data...|
|  1064.542299940903|203.8894319396973|22.85712617975521| 0.0|[0.05241657793521...|0.031245738184907168| 0.6967086747446842|22.297718511935173|128849052864|0.005392748172032213|ID18150/Day2/Data...|
|0.03

In [14]:
df.where(df.index.isin([300,400])).take(2)

[Row(time=10.40352990646771, position=303.7098174791827, original_velocity=22.97824965122394, pump=0.0, neural_data=[0.0013656221926794387, 0.0, 0.0017444707045797259, 0.000524658738868311, 0.002700238663237542, 0.0013478406617650762, 0.002971726018586196, 0.0013486453681252897, 0.000673954957164824, 0.0019533552695065737, 0.003248923283535987, 0.0019437470473349094, 0.006218665235792287, 0.007709247700404376, 0.0004077472403878346, 0.0014559410919900984, 0.013866361463442445, 0.0009866170221357606, 0.0025787973136175424, 0.0029110574687365443, 6.888731149956584e-05, 0.003609489751397632, 0.003379365080036223, 0.003814671259988245, 0.002384068808169104, 0.0, 0.002079587458865717, 0.0015136712609091774, 0.006746725179255009, 0.011375446803867817, 0.0, 0.0037978731270413846, 0.020811268594115973, 0.0004625998990377411, 0.0018709935975493863, 0.002631615920108743, 0.002259979388327338, 0.01983196916989982, 0.014820374548435211, 0.003057815949432552, 0.053889754228293896, 0.050392328063026

In [13]:
delta = 5000
def get_x_ticks(L:int):
    x_ticks = np.arange(0,L,delta)
    x_tick_labels = [f"{t[i]/100:.2f}" for i in x_ticks]
    return x_ticks, x_tick_labels

In [14]:
split = int(0.2*len(X))
X_train, X_test = X[:-split], X[-split:]
y_train, y_test = df["Pump"].values[:-split], df["Pump"].values[-split:]
X_train.shape, X_test.shape

((52796, 709), (13198, 709))

In [15]:
# x_ticks, x_tick_labels = get_x_ticks(X.shape[0])
# fig, ax = plt.subplots(1, 1, figsize=(15, 6))

# ax = sns.heatmap(X.T, ax=ax, cmap="gray_r")
# ax.set_xticks(x_ticks)
# ax.set_xticklabels(x_tick_labels)

# v_bar = X_train.shape[0]
# ax.axvline(v_bar, color="red")

# ax.set_title("Train and Test split")

# plt.show()

# CEBRA Encoder

## Dataset Preparation

In [16]:
with device:
    X_train_tensor = torch.from_numpy(X_train).type(torch.FloatTensor)
    X_test_tensor = torch.from_numpy(X_test).type(torch.FloatTensor)
    y_train_tensor = torch.from_numpy(y_train).type(torch.LongTensor)
    y_test_tensor = torch.from_numpy(y_test).type(torch.LongTensor)

In [40]:
X_train_tensor.dtype

torch.float32

# Models

In [17]:
class ModelPipeline:
    def __init__(self, model_name, X_train_tensor, y_train_tensor, X_test_tensor, y_test_tensor, num_units, latent_dimension = 8):
        self.model_name = model_name
        self.X_train_tensor = X_train_tensor
        self.y_train_tensor = y_train_tensor
        self.X_test_tensor = X_test_tensor
        self.y_test_tensor = y_test_tensor
        self.latent_dimension = latent_dimension
        self.num_units = num_units
        self.train_dataset = TensorDataset(
            neural = X_train_tensor,
            discrete = y_train_tensor,
        )
        self.test_dataset = TensorDataset(
            neural = X_test_tensor,
            discrete = y_test_tensor
        )
        with device:
            self.model = cebra.models.init(
                name = model_name,
                num_neurons = self.train_dataset.neural.shape[1],
                num_units = num_units,
                num_output = latent_dimension
            )
            self.train_dataset.configure_for(self.model)
            self.test_dataset.configure_for(self.model)
    def train_embedding(self, learning_rate, batch_size, steps = 1000, verbose = True):
        self.batch_size = batch_size
        self.steps = steps
        self.learning_rate = learning_rate
        self.steps = steps
        with device:
            criterion = cebra.models.criterions.LearnableCosineInfoNCE()
            optimizer = torch.optim.Adam(
                list(self.model.parameters()) + list(criterion.parameters()),
                lr = learning_rate
            )
            self.embedding_solver = cebra.solver.SingleSessionSolver(
                model = self.model,
                criterion = criterion,
                optimizer = optimizer,
                tqdm_on = verbose
            )
            train_loader = cebra.data.single_session.DiscreteDataLoader(
                dataset = self.train_dataset,
                num_steps = steps,
                batch_size = batch_size,
                prior = "empirical"
            )
            self.embedding_solver.fit(loader=train_loader)
    def train_decoder(self, verbose = True):
        with device:
            self.simple_train_dataloader = torch.utils.data.DataLoader(
                SimpleTensorDataset(
                    data = self.X_train_tensor.type(torch.FloatTensor),
                    labels = self.y_train_tensor.type(torch.FloatTensor),
                    offset = self.embedding_solver.model.get_offset(),
                    device = device
                ),
                batch_size = self.batch_size,
                shuffle = True
            )
            self.binaryClassifier = torch.nn.Sequential(
                self.model,
                torch.nn.Linear(self.latent_dimension,self.latent_dimension),
                torch.nn.GELU(),
                torch.nn.Linear(self.latent_dimension,1),
                torch.nn.GELU(),
                torch.nn.Linear(1,1) #Logit output
            ).to(device)
            self.decoder_solver = SupervisedNNSolver(
                model = self.binaryClassifier,
                criterion = torch.nn.BCEWithLogitsLoss(),
                optimizer = torch.optim.Adam(self.binaryClassifier.parameters(), lr = self.learning_rate)
            )
            self.decoder_solver.fit(self.simple_train_dataloader, num_steps = self.steps)
    def train(self, learning_rate, batch_size, steps, verbose = True):
        self.train_embedding(learning_rate, batch_size, steps, verbose)
        self.train_decoder(verbose = verbose)
    def score(self, verbose = True):
        with device:
            test_loader = cebra.data.single_session.DiscreteDataLoader(
                dataset = self.test_dataset,
                num_steps = self.steps,
                batch_size = self.batch_size,
                prior = "empirical"
            )
            self.embedding_score = self.embedding_solver.validation(test_loader)
            self.simple_test_dataloader = torch.utils.data.DataLoader(
                SimpleTensorDataset(
                    data = self.X_test_tensor.type(torch.FloatTensor),
                    labels = self.y_test_tensor.type(torch.FloatTensor),
                    offset = self.embedding_solver.model.get_offset(),
                    device = device
                    ),
                batch_size = self.batch_size,
                shuffle = True
                )
            self.decoder_score = self.decoder_solver.validation(self.simple_test_dataloader)['total']
            try:
                # Fit an exponential decay to the history and get the R2 value
                def exp_decay(x, a, b, c):
                    return a * np.exp(-b * x) + c

                # Fit the exponential decay to the solver history
                x_data = np.arange(len(self.embedding_solver.history))
                y_data = self.embedding_solver.history
                y0 = y_data[0]
                yf = y_data[-1]
                popt, _ = curve_fit(exp_decay, x_data, y_data, p0=(y0 - yf, 1e-6, yf))

                # Calculate the R2 value
                y_pred = exp_decay(x_data, *popt)
                r2 = r2_score(y_data, y_pred)
                if verbose:
                    print(f"Expontial decay R2: {r2}, embedding score: {self.embedding_score}, decoder score: {self.decoder_score}")
                return -r2 + self.embedding_score + 2*self.decoder_score
            except:
                return float("inf")
        

In [46]:
pipeline = ModelPipeline(
    "offset36-model",
    X_train_tensor=X_train_tensor,
    y_train_tensor=y_train_tensor,
    X_test_tensor=X_test_tensor,
    y_test_tensor=y_test_tensor,
    latent_dimension=8,
    num_units=2
)
pipeline.train(learning_rate = 1e-3, batch_size = 12, steps = 10)
pipeline.score()

pos: -0.9850 neg:  3.4666 total:  2.4816 temperature:  0.9968: 100%|██████████| 10/10 [00:01<00:00,  5.97it/s]
100%|██████████| 10/10 [00:00<00:00, 21.33it/s]


Expontial decay R2: 0.1465373380689179, embedding score: 2.484728240966797, decoder score: 0.8064513206481934


3.951093544194266

In [ ]:
models_to_try = ('offset10-model', 'offset36-model', 'offset1-model')
models_to_try

('offset10-model', 'offset36-model', 'offset1-model', 'offset5-model')

In [35]:
def experiment(trial):
    model_name = trial.suggest_categorical("model_name", models_to_try)
    num_units = trial.suggest_int("num_units", 1, X_train.shape[1])
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_int("batch_size", 50, 256, log=True)
    model = ModelPipeline(
        model_name,
        X_train_tensor=X_train_tensor,
        y_train_tensor=y_train_tensor,
        X_test_tensor=X_test_tensor,
        y_test_tensor=y_test_tensor,
        num_units=num_units,
        latent_dimension=8
    )
    model.train(learning_rate, batch_size, 500)
    score = model.score()
    
    # Save the best model to disk
    if not hasattr(experiment, "best_score") or score < experiment.best_score:
        experiment.best_score = score
        experiment.pipeline = model
        torch.save(model.model.state_dict(), "./data/models/best_model.pth")
    
    return score

In [36]:
study = optuna.create_study(storage="sqlite:///data/optuna.db", study_name="cebra_offsets", direction="minimize", load_if_exists=True)
study.optimize(experiment, n_trials=100)

study.best_params

[I 2024-11-29 15:16:13,326] Using an existing study with name 'cebra_offsets' instead of creating a new one.
pos: -0.9735 neg:  5.9531 total:  4.9797 temperature:  0.9370: 100%|██████████| 500/500 [04:10<00:00,  2.00it/s]
[W 2024-11-29 15:20:34,993] Trial 1 failed with parameters: {'model_name': 'offset5-model', 'num_units': 229, 'learning_rate': 0.00016581411988856763, 'batch_size': 163} because of the following error: RuntimeError('Input type (double) and bias type (float) should be the same').
Traceback (most recent call last):
  File "/home/icaro/Documents/doctorate/dimensionality-reduction/.venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_91719/3262429375.py", line 15, in experiment
    model.train(learning_rate, batch_size, 500)
  File "/tmp/ipykernel_91719/2048962979.py", line 79, in train
    self.train_decoder(verbose = verbose)
  File "/tmp/ipykernel

RuntimeError: Input type (double) and bias type (float) should be the same